In [173]:
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
import copy
import numpy as np
import json
import pickle
import random

Entrez.email = "fabian.spoendlin@exeter.ox.ac.uk"


In [ ]:
with open('CoV-AbDab_181021.csv', 'r') as f:
    CovAbDab = pd.read_csv(f)   

CovAbDab['VH or VHH'].replace(to_replace='ND', value='no sequence available', inplace=True)
CovAbDab['VL'].replace(to_replace='ND', value='no sequence available', inplace=True)

In [2]:
handle = Entrez.efetch(db='pubmed', id='32571838')
record = handle.read()


In [3]:
related = Entrez.read(Entrez.elink(db='protein',dbfrom='pubmed', id='32571838'))
for linksetdb in related[0]["LinkSetDb"]:
    print(linksetdb["DbTo"], linksetdb["LinkName"], len(linksetdb["Link"]))


protein pubmed_protein 9


In [4]:
related = Entrez.read(Entrez.elink(db='nucleotide',dbfrom='pubmed', id='32571838'))
for linksetdb in related[0]["LinkSetDb"]:
    print(linksetdb["DbTo"], linksetdb["LinkName"], len(linksetdb["Link"]))
related[0]

{'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'pubmed', 'IdList': ['32571838']}

In [5]:
#search nucleotides and check how many of the sequences are in Covab Dab
handle = Entrez.esearch(db='nucleotide', term='anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]', retmax='50')
record = Entrez.read(handle)

nucleotides = []

for ID in record['IdList']:
    nucelotide = Entrez.efetch(db="nucleotide", id=ID, rettype="gb", retmode="text")
    nucelotide_formated = SeqIO.read(nucelotide,'genbank')
    nucleotides.append(nucelotide_formated)


In [6]:
summary_n = []

for entry in nucleotides:

    lenght_sequence = len(entry.seq)
    remove_bases = lenght_sequence % 3
    if remove_bases == 0:
        aa_seq = str(entry.seq.translate()) # problems if nucleotide sequence is not in correct frame
    else:
        aa_seq = str(entry.seq[:-remove_bases].translate())

    containes = []
    for VH in CovAbDab['VH or VHH']:
        if VH in aa_seq:
            containes.append('heavy chain')
    for VL in CovAbDab['VL']:
        if str(VL) in aa_seq: # nan in VL that is formated as float
            containes.append('light chain')
    
    if 'heavy chain' in containes:
        summary_n.append('heavy chain')
        
    elif 'light chain' in containes:
        summary_n.append('light chain') 

    else:
        summary_n.append('not in covab-dab')
        
print(summary_n.count('not in covab-dab'),len(summary_n))


17 50


In [62]:
#search proteins and check how many of the sequences are in Covab Dab
handle = Entrez.esearch(db='protein', term='anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]', retmax='50')
record = Entrez.read(handle)

proteins = []

for ID in record['IdList']:
    protein = Entrez.efetch(db="protein", id=ID, rettype="gb", retmode="text")
    protein_formated = SeqIO.read(protein,'genbank')
    proteins.append(protein_formated)

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(


In [66]:
summary_p = []

for entry in proteins:

    aa_seq = str(entry.seq)

    containes = []
    for VH in CovAbDab['VH or VHH']:
        if VH in aa_seq:
            containes.append('heavy chain')
    for VL in CovAbDab['VL']:
        if str(VL) in aa_seq: # nan in VL that is formated as float
            containes.append('light chain')
    
    if 'heavy chain' in containes:
        summary_p.append('heavy chain')
        
    elif 'light chain' in containes:
        summary_p.append('light chain') 

    else:
        summary_p.append('not in covab-dab')

print(summary_p.count('not in covab-dab'),len(summary_p))

13 50


In [67]:
#nucelotide and protein database contains sequence of whole chain not just VL and VH, thus this comparison does not work
summary_2_n = []

for entry in nucleotides:

    aa_seq = str(entry.seq.translate())
   
    if aa_seq in CovAbDab['VH or VHH']:
        summary_2_n.append('heavy chain')

    elif aa_seq in str(CovAbDab['VL']):
        summary_2_n.append('light chain') 

    else:
        summary_2_n.append('not in covab-dab')

print(summary_2_n.count('not in covab-dab'),len(summary_2_n))


summary_2_p = []

for entry in proteins:

    aa_seq = str(entry.seq)
   
    if aa_seq in CovAbDab['VH or VHH']:
        summary_2_p.append('heavy chain')

    elif aa_seq in str(CovAbDab['VL']):
        summary_2_p.append('light chain') 

    else:
        summary_2_p.append('not in covab-dab')

print(summary_2_p.count('not in covab-dab'),len(summary_2_p))

50 50
50 50


/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/Seq.py:2979: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


## Search genbank protein database with certain keywords and evaluate suitability of keywords by comparing the found sequences with covab dab

In [143]:
# specify the terms used for the search
search = 'anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]'

# search protein data base with keywords and find out how many entries are found
handle = Entrez.esearch(db='protein', term='anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]', retmax='2')
record = Entrez.read(handle)
number_of_entries = int(record['Count'])
print('number of entries:', number_of_entries)

number of entries: 2685


In [160]:
#Entrez.api_key = "MyAPIkey"

# access ids of all entries obtained for a search
handle = Entrez.esearch(db='protein', term=search, retmax=number_of_entries)
record = Entrez.read(handle)

# approximately 1 entry fetched per second
# randomly sample the entries to reduce time needed for program to run
number_of_samples = 200
sampled_ids = random.sample(record['IdList'], number_of_samples)

# fetch the randomly selected entries and save in list
proteins = []
count = 0

for ID in sampled_ids:
    protein = Entrez.efetch(db="protein", id=ID, rettype="gb", retmode="text")
    protein_formated = SeqIO.read(protein,'genbank')
    proteins.append(protein_formated)

    count += 1

    if count % 10 == 0:
        print('number of entires fetched:', count)

protein_handle =  Entrez.efetch(db="protein", id=sampled_ids, rettype="gb", retmode="xml")
#proteins = SeqIO.read(protein_handle,'genbank')
#or
#proteins = SeqIO.parse(protein_handle,'gb')
proteins = Entrez.read(protein_handle) # ---> this is much quicker

In [122]:
# copy covabdab dataframe to add stats
CovAbDab_stats = copy.deepcopy(CovAbDab)

# add columns to count how many times a certain VH or VL was found in the genbank
CovAbDab_stats['VH_found'] = 0
CovAbDab_stats['VL_found'] = 0

sequences_not_in_covabdab = 0
iteration_count = 0

# loop throught the feched proteins
for entry in proteins:

    iteration_count += 1

    #s eq to string
    aa_seq = str(entry.seq)

    sequence_found = False

    # loop throught covab dab entries
    for i in range(len(CovAbDab_stats)):
        
        # in case VH is in covab dab increase the VH count of this entry by 1
        if CovAbDab_stats.iloc[i,8] in aa_seq:
            CovAbDab_stats.iloc[i,-2] = CovAbDab_stats.iloc[i,-2] + 1
            sequence_found = True

        # in case VL is in covab dab increase the VL count of this entry by 1
        if str(CovAbDab_stats.iloc[i,9]) in aa_seq:
            CovAbDab_stats.iloc[i,-1] = CovAbDab_stats.iloc[i,-1] + 1
            sequence_found = True

    # sequence that has no match with vh or vl is counted as a not found sequence
    if not sequence_found:
        sequences_not_in_covabdab += 1

sequences_in_covabdab = iteration_count - sequences_not_in_covabdab


In [124]:
#print summary statistics
multiplicator = number_of_entries / number_of_samples # factor to adjust statistics by the fraction of sampled entires

if True:
    print('total sequences assessed:', iteration_count * multiplicator)
    print('number of genbank sequences not in covab dab:', sequences_not_in_covabdab * multiplicator)
    print('number of genbank sequences found in covab dab:', sequences_in_covabdab * multiplicator)
    print('match rate:', sequences_in_covabdab / iteration_count)
    # if the total number of counts in VH and VL columns is higher than the genbank sequences that have a match in covab dab
    # then a genbank sequence must have several matches in covab dab
    print('number of genebank sequences that have multiple matches in covab dab:', (sum(CovAbDab_stats['VH_found'])+sum(CovAbDab_stats['VL_found'])-sequences_in_covabdab) * multiplicator)
    # if the number of genbank sequences with a match in covab dab is higher than the number of covab dab VH and VLs that were found
    # then a number of genbank sequences must have matched to the same covab dab sequenc
    print('number of genebank entries with non unique match in covab dab:', (sequences_in_covabdab - (len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0)]) + len(CovAbDab_stats.loc[(CovAbDab_stats['VL_found'] > 0)]))) * multiplicator)
    print('-------------')
    print('total sequences in covab dab:', len(CovAbDab_stats))
    print('number of Covab Dab VH sequences found:', len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0)]) * multiplicator)
    print('number of Covab Dab VL sequences found:', len(CovAbDab_stats.loc[(CovAbDab_stats['VL_found'] > 0)]) * multiplicator)
    VH_VL_pairings = len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0) & (CovAbDab_stats['VL_found'] > 0)]) * multiplicator
    print('number of Covab Dab VH VL pairings found:', VH_VL_pairings)
    VH_or_VL = len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0) | (CovAbDab_stats['VL_found'] > 0)]) * multiplicator
    print('number of Covab Dab entires where either VH or VL was found:', VH_or_VL)
    print('-------------')
    print('percentage of covab dab entries with pairing found:', VH_VL_pairings / len(CovAbDab_stats) * 100 )
    print('percentage of covab dab entries with either VL or VH found:', VH_or_VL / len(CovAbDab_stats) * 100)

total sequences assessed: 2685
number of genbank sequences not in covab dab: 409
number of genbank sequences found in covab dab: 2276
match rate: 0.8476722532588454
number of genebank sequences that have multiple matches in covab dab: 636
number of genebank entries with non unique match in covab dab: 13
-------------
total sequences in covab dab: 4198
number of Covab Dab VH sequences found: 1110
number of Covab Dab VL sequences found: 1153
number of Covab Dab VH VL pairings found: 1059
number of Covab Dab entires where either VH or VL was found: 1204
-------------
percentage of covab dab entries with pairing found: 25.226298237255833
percentage of covab dab entries with either VL or VH found: 28.680323963792283


In [131]:
# save stats of search perfomed
with open('data/search_stats.csv', 'r+') as fo:
    fo.read()
    fo.write(f'''{search} - samples: {number_of_samples}, 
                {iteration_count * multiplicator}, 
                {sequences_not_in_covabdab * multiplicator}, 
                {sequences_in_covabdab * multiplicator}, 
                {sequences_in_covabdab / iteration_count}, 
                {(sum(CovAbDab_stats["VH_found"]) + sum(CovAbDab_stats["VL_found"]) - sequences_in_covabdab) * multiplicator}, 
                {(sequences_in_covabdab - (len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0)])) + len(CovAbDab_stats.loc[(CovAbDab_stats["VL_found"] > 0)])) * multiplicator}, 
                {len(CovAbDab_stats)}, {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0) & (CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0) | (CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {VH_VL_pairings / len(CovAbDab_stats) * 100}, {VH_or_VL / len(CovAbDab_stats) * 100}''')

In [132]:
with open('data/search_stats.csv', 'r+') as fo:
    df = pd.read_csv('data/search_stats.csv')

df.head()

,search,total sequences assessed,number of genbank sequences not in covab dab,number of genbank sequences found in covab dab,match rate,number of genebank sequences that have multiple matches in covab dab,number of genebank entries with non unique match in covab dab,total sequences in covab dab,number of Covab Dab VH sequences found,number of Covab Dab VL sequences found,number of Covab Dab VH VL pairings found,number of Covab Dab entires where either VH or VL was found,percentage of covab dab entries with pairing found,percentage of covab dab entries with either VL or VH found
0,anti-sars-cov-2[All Fields] AND immunoglobulin...,2685,409,2276,0.847672,636,2319,4198,1110,1153,1059,1204,25.226298,28.680324


In [ ]:
# save fetched gene bank entries into pickle file
out_file_name = #change this f'data/{search}-{number_of_samples}.obj'
out_file = open(out_file_name, 'wb')
pickle.dump(proteins, out_file)
out_file.close()

In [116]:
# load results from previous search
in_file = open('data/anti-sars-cov-2_immunoglobulin.obj', 'wb')
proteins = pickle.load(in_file)
in_file.close()

'/Users/fabian/Desktop/SABS/Antibody project/code/auto-db-pipeline/genbank'

## Same function but faster

In [176]:
# specify the terms used for the search
search = 'anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]'

# search protein data base with keywords and find out how many entries are found
handle = Entrez.esearch(db='protein', term='anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]', retmax='2')
record = Entrez.read(handle)
number_of_entries = int(record['Count'])
print('number of entries:', number_of_entries)

number of entries: 2685


In [177]:
# access ids of all entries obtained for a search
handle = Entrez.esearch(db='protein', term=search, retmax=number_of_entries)
record = Entrez.read(handle)

# 25 searches per second
protein_handle =  Entrez.efetch(db="protein", id=record['IdList'], rettype="gb", retmode="xml")
proteins = Entrez.read(protein_handle)

In [178]:
# this runs slow, look at possible use of multiprocessing

# copy covabdab dataframe to add stats
CovAbDab_stats = copy.deepcopy(CovAbDab)

# add columns to count how many times a certain VH or VL was found in the genbank
CovAbDab_stats['VH_found'] = 0
CovAbDab_stats['VL_found'] = 0

sequences_not_in_covabdab = 0
iteration_count = 0

# loop throught the feched proteins
for entry in proteins:

    iteration_count += 1

    # seq to string
    aa_seq = str.upper(entry['GBSeq_sequence'])

    sequence_found = False

    # loop throught covab dab entries
    for i in range(len(CovAbDab_stats)):
        
        # in case VH is in covab dab increase the VH count of this entry by 1
        if CovAbDab_stats.iloc[i,8] in aa_seq:
            CovAbDab_stats.iloc[i,-2] = CovAbDab_stats.iloc[i,-2] + 1
            sequence_found = True

        # in case VL is in covab dab increase the VL count of this entry by 1
        if str(CovAbDab_stats.iloc[i,9]) in aa_seq:
            CovAbDab_stats.iloc[i,-1] = CovAbDab_stats.iloc[i,-1] + 1
            sequence_found = True

    # sequence that has no match with vh or vl is counted as a not found sequence
    if not sequence_found:
        sequences_not_in_covabdab += 1

sequences_in_covabdab = iteration_count - sequences_not_in_covabdab

In [179]:
#print summary statistics
if True:
    print('total sequences assessed:', iteration_count)
    print('number of genbank sequences not in covab dab:', sequences_not_in_covabdab)
    print('number of genbank sequences found in covab dab:', sequences_in_covabdab)
    print('match rate:', sequences_in_covabdab / iteration_count)
    # if the total number of counts in VH and VL columns is higher than the genbank sequences that have a match in covab dab
    # then a genbank sequence must have several matches in covab dab
    print('number of genebank sequences that have multiple matches in covab dab:', (sum(CovAbDab_stats['VH_found'])+sum(CovAbDab_stats['VL_found'])-sequences_in_covabdab))
    # if the number of genbank sequences with a match in covab dab is higher than the number of covab dab VH and VLs that were found
    # then a number of genbank sequences must have matched to the same covab dab sequenc
    print('number of genebank entries with non unique match in covab dab:', (sequences_in_covabdab - (len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0)]) + len(CovAbDab_stats.loc[(CovAbDab_stats['VL_found'] > 0)]))))
    print('-------------')
    print('total sequences in covab dab:', len(CovAbDab_stats))
    print('number of Covab Dab VH sequences found:', len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0)]))
    print('number of Covab Dab VL sequences found:', len(CovAbDab_stats.loc[(CovAbDab_stats['VL_found'] > 0)]))
    VH_VL_pairings = len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0) & (CovAbDab_stats['VL_found'] > 0)])
    print('number of Covab Dab VH VL pairings found:', VH_VL_pairings)
    VH_or_VL = len(CovAbDab_stats.loc[(CovAbDab_stats['VH_found'] > 0) | (CovAbDab_stats['VL_found'] > 0)])
    print('number of Covab Dab entires where either VH or VL was found:', VH_or_VL)
    print('-------------')
    print('percentage of covab dab entries with pairing found:', VH_VL_pairings / len(CovAbDab_stats) * 100 )
    print('percentage of covab dab entries with either VL or VH found:', VH_or_VL / len(CovAbDab_stats) * 100)

total sequences assessed: 2685
number of genbank sequences not in covab dab: 409
number of genbank sequences found in covab dab: 2276
match rate: 0.8476722532588454
number of genebank sequences that have multiple matches in covab dab: 636
number of genebank entries with non unique match in covab dab: 13
-------------
total sequences in covab dab: 4198
number of Covab Dab VH sequences found: 1110
number of Covab Dab VL sequences found: 1153
number of Covab Dab VH VL pairings found: 1059
number of Covab Dab entires where either VH or VL was found: 1204
-------------
percentage of covab dab entries with pairing found: 25.226298237255833
percentage of covab dab entries with either VL or VH found: 28.680323963792283


In [ ]:
# save stats of search perfomed
with open('data/search_stats.csv', 'r+') as fo:
    fo.read()
    fo.write(f'''{search} - samples: {number_of_samples}, 
                {iteration_count * multiplicator}, 
                {sequences_not_in_covabdab * multiplicator}, 
                {sequences_in_covabdab * multiplicator}, 
                {sequences_in_covabdab / iteration_count}, 
                {(sum(CovAbDab_stats["VH_found"]) + sum(CovAbDab_stats["VL_found"]) - sequences_in_covabdab) * multiplicator}, 
                {(sequences_in_covabdab - (len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0)])) + len(CovAbDab_stats.loc[(CovAbDab_stats["VL_found"] > 0)])) * multiplicator}, 
                {len(CovAbDab_stats)}, {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0) & (CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {len(CovAbDab_stats.loc[(CovAbDab_stats["VH_found"] > 0) | (CovAbDab_stats["VL_found"] > 0)]) * multiplicator}, 
                {VH_VL_pairings / len(CovAbDab_stats) * 100}, {VH_or_VL / len(CovAbDab_stats) * 100}''')

## Next tasks
1. optimise keywords to find more sequences
2. search nucleotide genbank and see if this has more papers
3. look at proteins found in genbank that are not in covab dab, are these false positives or relevant antibodies missing from covab dab

In [17]:
#get more infomrtion for each entry
handle = Entrez.esearch(db='nucleotide', term='anti-sars-cov-2[All Fields] AND immunoglobulin[All Fields]', retmax='10')
record = Entrez.read(handle)

genbank_entries_2 = []

for ID in record['IdList']:
    genbank_entries_2.append(Entrez.efetch(db="nucleotide", id=ID, rettype="gb", retmode="text").read())

print(genbank_entries_2[0])

LOCUS       MZ751050                 321 bp    mRNA    linear   ROD 17-AUG-2021
DEFINITION  Mus musculus clone 15G9/10D2 anti-SARS-CoV-2 spike protein
            immunoglobulin light chain variable region mRNA, partial cds.
ACCESSION   MZ751050
VERSION     MZ751050.1
KEYWORDS    .
SOURCE      Mus musculus (house mouse)
  ORGANISM  Mus musculus
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Glires; Rodentia; Myomorpha;
            Muroidea; Muridae; Murinae; Mus; Mus.
REFERENCE   1  (bases 1 to 321)
  AUTHORS   Zhang,G., Wang,A. and Jiang,M.
  TITLE     Epitope profiling reveals the critical antigenic determinants in
            SARSCoV-2 RBD-based antigen
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 321)
  AUTHORS   Zhang,G., Wang,A. and Jiang,M.
  TITLE     Direct Submission
  JOURNAL   Submitted (09-AUG-2021) College of Animal Science and Veterinary
            Medicine, Henan Agricultural Univers